# AdaBoost 自适应提升算法

## 算法原理

AdaBoost (Adaptive Boosting) 是一种迭代式的集成学习算法，其核心思想是：

1. **样本权重调整**：每轮迭代后，增加被错误分类样本的权重，减少被正确分类样本的权重
2. **弱学习器组合**：将多个弱学习器（通常是决策树桩）通过加权投票组合成强学习器
3. **串行训练**：每个弱学习器都试图修正前一个学习器的错误

## 数学表达

对于 $M$ 个弱学习器，最终预测为：

$$H(x) = \text{sign}\left(\sum_{m=1}^{M} \alpha_m h_m(x)\right)$$

其中 $\alpha_m$ 是第 $m$ 个弱学习器的权重，$h_m(x)$ 是弱学习器的预测。

## 关键参数

- `n_estimators`: 弱学习器数量，过多可能导致过拟合
- `learning_rate`: 学习率，控制每个弱学习器的贡献度
- `estimator`: 基学习器（默认为决策树桩 max_depth=1）

In [ ]:
# 导入必要的库
import numpy as np
from sklearn.datasets import load_iris
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report

# 设置随机种子以保证结果可复现
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

## 1. 数据准备

使用经典的鸢尾花（Iris）数据集进行演示。

In [ ]:
# 加载数据集
iris = load_iris()
X, y = iris.data, iris.target

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

print(f"训练集大小: {X_train.shape[0]}")
print(f"测试集大小: {X_test.shape[0]}")
print(f"特征数量: {X.shape[1]}")
print(f"类别数量: {len(np.unique(y))}")

## 2. AdaBoost 模型训练

使用决策树桩（深度为1的决策树）作为基学习器，这是 AdaBoost 最常见的配置。

In [ ]:
# 定义基学习器：决策树桩
base_estimator = DecisionTreeClassifier(max_depth=1, random_state=RANDOM_STATE)

# 创建 AdaBoost 分类器
# 注意：sklearn 1.6+ 中 SAMME.R 已废弃，使用 SAMME 算法
ada_clf = AdaBoostClassifier(
    estimator=base_estimator,    # 基学习器
    n_estimators=200,            # 弱学习器数量
    learning_rate=0.5,           # 学习率
    algorithm='SAMME',           # 使用 SAMME 算法
    random_state=RANDOM_STATE
)

# 训练模型
ada_clf.fit(X_train, y_train)

# 预测
y_pred = ada_clf.predict(X_test)

# 评估模型
train_score = ada_clf.score(X_train, y_train)
test_score = ada_clf.score(X_test, y_test)

print(f"训练集准确率: {train_score:.4f}")
print(f"测试集准确率: {test_score:.4f}")

## 3. 交叉验证评估

使用 5 折交叉验证来更可靠地评估模型性能。

In [ ]:
# 交叉验证
cv_scores = cross_val_score(ada_clf, X, y, cv=5, scoring='accuracy')

print(f"交叉验证分数: {cv_scores}")
print(f"平均准确率: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

## 4. 学习曲线分析

分析不同数量的弱学习器对模型性能的影响。

In [ ]:
# 分析弱学习器数量对性能的影响
n_estimators_range = [10, 50, 100, 200, 300]
results = []

for n_est in n_estimators_range:
    clf = AdaBoostClassifier(
        estimator=DecisionTreeClassifier(max_depth=1, random_state=RANDOM_STATE),
        n_estimators=n_est,
        learning_rate=0.5,
        algorithm='SAMME',
        random_state=RANDOM_STATE
    )
    clf.fit(X_train, y_train)
    train_acc = clf.score(X_train, y_train)
    test_acc = clf.score(X_test, y_test)
    results.append((n_est, train_acc, test_acc))
    print(f"n_estimators={n_est:3d}: 训练准确率={train_acc:.4f}, 测试准确率={test_acc:.4f}")

## 5. 单元测试验证

验证模型的基本功能是否正常。

In [ ]:
# 单元测试
def test_adaboost():
    """AdaBoost 模型功能测试"""
    # 测试1: 模型应该能够训练
    assert hasattr(ada_clf, 'estimators_'), "模型未正确训练"
    
    # 测试2: 弱学习器数量应该正确
    assert len(ada_clf.estimators_) == 200, "弱学习器数量不正确"
    
    # 测试3: 预测结果形状应该正确
    assert y_pred.shape == y_test.shape, "预测结果形状不正确"
    
    # 测试4: 准确率应该在合理范围内
    assert test_score >= 0.8, f"测试集准确率过低: {test_score}"
    
    # 测试5: 交叉验证应该返回正确数量的分数
    assert len(cv_scores) == 5, "交叉验证折数不正确"
    
    print("所有测试通过!")

test_adaboost()

## 总结

### AdaBoost 的优势
- 不易过拟合（在某些条件下）
- 可以使用各种弱学习器
- 理论基础扎实

### AdaBoost 的局限
- 对噪声和异常值敏感
- 训练是串行的，无法并行化
- 需要足够的数据量

### 调参建议
1. 先固定 `learning_rate=1.0`，调整 `n_estimators`
2. 再降低 `learning_rate`，相应增加 `n_estimators`
3. 使用交叉验证选择最佳参数组合